In [1]:
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


Представим, что международное круизное агентство Carnival Cruise Line решило себя разрекламировать с помощью баннеров и обратилось для этого к вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с вами сотрудничать.

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры — легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях — Foursquare.

Нас будет интересовать файл checkins.dat. Открыв его, увидим следующую структуру:  
Для удобной работы с этим документом преобразуем его к формату csv, удалив строки, не содержащие координат — они неинформативны для нас:

id,user_id,venue_id,latitude,longitude,created_at

984222,15824,5222,38.8951118,-77.0363658,2012-04-21T17:43:47

984234,44652,5222,33.800745,-84.41052,2012-04-21T17:43:43

984291,105054,5222,45.5234515,-122.6762071,2012-04-21T17:39:22

In [2]:
data = pd.read_csv("/home/storm/Datasets/fsq/umn_foursquare_datasets/checkins.dat", sep="|")[3:]

/home/storm/anaconda3/envs/ipykernel_py2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data.columns = [i.strip() for i in data.columns]
data["latitude"].replace(' ' * 19, np.nan, inplace = True)
data["longitude"].replace(' ' * 19, np.nan, inplace = True)
data.dropna(inplace=True)

sub_data = data[:100000]
del data

print(sub_data.shape)
sub_data.head()

(100000, 6)


,id,user_id,venue_id,latitude,longitude,created_at
4,984234,44652.0,5222.0,33.800745,-84.41052,2012-04-21 17:43:43
8,984291,105054.0,5222.0,45.5234515,-122.6762071,2012-04-21 17:39:22
10,984318,2146539.0,5222.0,40.764462,-111.904565,2012-04-21 17:35:46
11,984232,93870.0,380645.0,33.4483771,-112.0740373,2012-04-21 17:38:18
12,984483,1030290.0,955969.0,32.2217429,-110.926479,2012-04-21 17:58:54


Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Эта задача — хороший повод познакомиться с алгоритмом MeanShift, который мы обошли стороной в основной части лекций. Его описание при желании можно посмотреть в sklearn user guide, а чуть позже появится дополнительное видео с обзором этого и некоторых других алгоритмов кластеризации. Используйте MeanShift, указав bandwidth=0.1, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

In [4]:
from sklearn.cluster import MeanShift

longlat_ = sub_data[["latitude", "longitude"]]
cluster = MeanShift(bandwidth=0.1)
cluster.fit(longlat_)

MeanShift(bandwidth=0.1, bin_seeding=False, cluster_all=True, min_bin_freq=1,
     n_jobs=None, seeds=None)

Некоторые из получившихся кластеров содержат слишком мало точек — такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.

При желании увидеть получившиеся результаты на карте можно передать центры получившихся кластеров в один из инструментов визуализации. Например, сайт mapcustomizer.com имеет функцию Bulk Entry, куда можно вставить центры полученных кластеров в формате:  

38.8951118,-77.0363658

33.800745,-84.41052

45.5234515,-122.6762071


  
  
Как мы помним, 20 баннеров надо разместить близ офисов компании. Найдем на Google Maps по запросу Carnival Cruise Line адреса всех офисов:

33.751277, -118.188740 (Los Angeles)

25.867736, -80.324116 (Miami)

51.503016, -0.075479 (London)

52.378894, 4.885084 (Amsterdam)

39.366487, 117.036146 (Beijing)

-33.868457, 151.205134 (Sydney)

In [5]:
centers = cluster.cluster_centers_
cities = np.array([[33.751277, -118.188740], 
                   [25.867736, -80.324116],
                   [51.503016, -0.075479],
                   [52.378894, 4.885084],
                   [39.366487, 117.036146], 
                   [-33.868457, 151.205134]])

labels = cluster.labels_
unique_labels = np.unique(cluster.labels_)

n_clusters = len(unique_labels)
count_labels = dict()

In [6]:
for i in range(len(labels)):
    if count_labels.has_key(labels[i]):
        count_labels[labels[i]] += 1
    else:
        count_labels[labels[i]] = 1

In [7]:
int_labels = [l for l in unique_labels if count_labels[l] > 16]
geo_pts = np.array([centers[i] for i in range(len(centers)) if int_labels.count(labels[i]) > 0])

In [8]:
best_dist = list()
for g in geo_pts:
    dist = np.inf
    for c in cities:
        c_dist = abs(g[0]-c[0]) + abs(g[1] - c[1])
        if dist > c_dist:
            dist = c_dist
    best_dist.append(dist)

best_dist = np.array(best_dist)
indexes = np.argsort(best_dist)[:20]

In [9]:
for data in geo_pts[indexes]:
    print(data)

[-33.86063043 151.20477593]
[52.37296399  4.89231722]
[ 25.84567226 -80.3188906 ]
[51.50299126 -0.12553729]
[  33.80987796 -118.14892381]
[ 25.78581242 -80.21793804]
[ 25.70534972 -80.28342874]
[-34.00190615 151.12806905]
[  33.97257482 -118.16837067]
[-33.9522629 151.0321372]
[ 26.01009825 -80.19999059]
[  33.88832534 -118.04892817]
[ 26.13884379 -80.33434684]
[51.50647877 -0.36517727]
[  33.87298601 -118.36209115]
[51.42676329 -0.30373207]
[51.5741517  0.1838708]
[  33.81730643 -117.89124917]
[52.2644  4.6347]
[  34.06039755 -118.24870903]


In [10]:
with open("ans.txt", "w") as fout:
    fout.write("-33.86063043 151.20477593")